In [1]:
pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -U pycryptodome


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

2024-03-04 23:06:52.004381: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-04 23:06:52.005809: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-04 23:06:52.031289: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 23:06:52.031317: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 23:06:52.031964: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [4]:
dataset = tf.keras.preprocessing.image_dataset_from_directory("Dataset_BP", shuffle = True, image_size  = (256,256), batch_size = 32)

Found 2475 files belonging to 2 classes.


In [5]:
class_names = dataset.class_names
class_names

['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy']

In [6]:
dataset = dataset.shuffle(10000, seed = 12)

In [7]:
train_ds = dataset.take(60)
len(train_ds)

60

In [8]:
left_ds= dataset.skip(54)
len(left_ds)

24

In [9]:
validation_ds = left_ds.take(10)
len(validation_ds)

10

In [10]:
test_ds= left_ds.skip(10)
len(test_ds)

14

In [11]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
val_ds = validation_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)

In [12]:
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(256,256),
    layers.experimental.preprocessing.Rescaling(1.0/255),
])


In [13]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
])

In [19]:
bp_model = models.Sequential([
    resize_and_rescale,
    data_augmentation,
    
    layers.Conv2D(32, (3,3), activation ='relu', input_shape = (32,256,256,3), padding = "same"),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation ='relu',padding = "same"),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation ='relu',padding = "same"),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation ='relu',padding = "same"),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation ='relu',padding = "same"),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation ='relu',padding = "same"),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation ='relu',padding = "same"),
    layers.MaxPooling2D(2,2),
    
    layers.Flatten(),
    layers.Dense(64,activation = 'relu'),
    layers.Dense(2, activation = 'softmax')
])

bp_model.build(input_shape = (32,256,256,3))

In [20]:
bp_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [21]:
bp_model.fit(
    train_ds,
    epochs = 10,
    batch_size = 32,
    verbose = 1,
    validation_data= val_ds,
)

Epoch 1/10
60/60 [==============================] - 32s 519ms/step - loss: 0.6788 - accuracy: 0.5922 - val_loss: 0.6652 - val_accuracy: 0.6187
Epoch 2/10
60/60 [==============================] - 33s 548ms/step - loss: 0.4830 - accuracy: 0.8089 - val_loss: 0.5386 - val_accuracy: 0.6438
Epoch 3/10
60/60 [==============================] - 33s 545ms/step - loss: 0.2222 - accuracy: 0.9245 - val_loss: 0.2591 - val_accuracy: 0.9187
Epoch 4/10
60/60 [==============================] - 33s 545ms/step - loss: 0.1453 - accuracy: 0.9630 - val_loss: 0.1273 - val_accuracy: 0.9500
Epoch 5/10
60/60 [==============================] - 33s 545ms/step - loss: 0.1026 - accuracy: 0.9693 - val_loss: 0.0587 - val_accuracy: 0.9844
Epoch 6/10
60/60 [==============================] - 33s 545ms/step - loss: 0.0695 - accuracy: 0.9812 - val_loss: 0.0616 - val_accuracy: 0.9937
Epoch 7/10
60/60 [==============================] - 33s 544ms/step - loss: 0.0558 - accuracy: 0.9870 - val_loss: 0.0296 - val_accuracy: 0.9875

In [22]:
bp_model.evaluate(test_ds)

14/14 [==============================] - 2s 143ms/step - loss: 0.0149 - accuracy: 1.0000


[0.014872903935611248, 1.0]

In [23]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)
    
    predictions = model.predict(img_array)
    
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100*(np.max(predictions[0])), 2)
    return predicted_class, confidence

In [24]:
bp_model.save(f"../Models/bell_pepper")

INFO:tensorflow:Assets written to: ../Models/bell_pepper/assets


INFO:tensorflow:Assets written to: ../Models/bell_pepper/assets
